In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime

W0509 09:49:02.001796 140149597505280 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [3]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [4]:
!mkdir bert_noun_number

mkdir: cannot create directory ‘bert_noun_number’: File exists


In [5]:
OUTPUT_DIR = "bert_noun_number"

In [6]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'bert_noun_number'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: bert_noun_number *****


# Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [7]:
from tensorflow import keras
import os
import re

In [8]:
Conventional = ["Generic correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Choice of tense", "Tense form", "Voice", "Modals", "Verb pattern", "Intransitive verb", "Transitive verb", "Reflexive verb", "Verb with as", "Ambitransitive verb", "Two verbal forms in the predicate", "Verb + Infinitive", "Verb + Gerund", "Verb + Infinitive OR Gerund", "Verb + Bare Infinitive", "Verb + Object/Addressee + Bare Infinitive", "Infinitive Restoration Alternation", "Verb + Participle", "Get + participle", "Complex-object verb", "Verbal idiom", "Prepositional or phrasal verb", "Dative verb with alternation", "Verb followed by a clause", "Verb + that/WH + Clause", "Verb + if/whether + clause", "Verb + that + Subjunctive clause", "Verb + it + Conj + Clause", "Participial construction", "Infinitive construction", "Gerund phrase", "Nouns", "Countable/uncountable", "Prepositional noun", "Possessive form of a noun", "Noun as an attribute", "Noun + Infinitive", "Noun number", "Prepositions", "Conjunctions", "Adjectives", "Comparative degree of adjectives", "Superlative degree of adjectives", "Prepositional adjective", "Adjective as a collective noun", "Adverbs", "Comparative degree of adverbs", "Superlative degree of adverbs", "Prepositional adverb", "Numerals", "Pronouns", "Agreement", "Word order", "Standard word order", "Emphatic shift", "Cleft sentence", "Interrogative word order", "Incomplete sentence", "Exclamation", "Title structure", "Note structure", "Conditionals", "Attributes", "Relative clause", "Defining relative clause", "Non-defining relative clause", "Coordinate relative clause", "Attributive participial construction", "Parallel constructions", "Negation", "Comparative construction", "Numerical comparison", "Confusion of structures", "Vocabulary", "Word choice", "Choice of lexical item", "Words often confused", "Choice of a part of lexical item", "Absence of certain components of a collocation", "Redundant word(s)", "Word formation", "Derivational affixes", "Formational suffix", "Formational prefix", "Confusion of categories", "Compound word", "Discourse", "Referential device", "Coherence", "Linking device", "Inappropriate register", "Absence of a component in clause or sentence", "Redundant component in clause or sentence", "Absence of necessary explanation or detail", "Deletion"]
Tags = ["Correction", "Punctuation", "Spelling", "Capitalisation", "Grammar", "Determiners", "Articles", "Quantifiers", "Verbs", "Tense", "Tense_choice", "Tense_form", "Voice", "Modals", "Verb_pattern", "Intransitive", "Transitive", "Reflexive_verb", "Presentation", "Ambitransitive", "Two_in_a_row", "Verb_Inf", "Verb_Gerund", "Verb_Inf_Gerund", "Verb_Bare_Inf", "Verb_object_bare", "Restoration_alter", "Verb_part", "Get_part", "Complex_obj", "Verbal_idiom", "Prepositional_verb", "Dative", "Followed_by_a_clause", "that_clause", "if_whether_clause", "that_subj_clause", "it_conj_clause", "Participial_constr", "Infinitive_constr", "Gerund_phrase", "Nouns", "Countable_uncountable", "Prepositional_noun", "Possessive", "Noun_attribute", "Noun_inf", "Noun_number", "Prepositions", "Conjunctions", "Adjectives", "Comparative_adj", "Superlative_adj", "Prepositional_adjective", "Adj_as_collective", "Adverbs", "Comparative_adv", "Superlative_adv", "Prepositional_adv", "Numerals", "Pronouns", "Agreement_errors", "Word_order", "Standard", "Emphatic", "Cleft", "Interrogative", "Abs_comp_clause", "Exclamation", "Title_structure", "Note_structure", "Conditionals", "Attributes", "Relative_clause", "Defining", "Non_defining", "Coordinate", "Attr_participial", "Lack_par_constr", "Negation", "Comparative_constr", "Numerical", "Confusion_of_structures", "Vocabulary", "Word_choice", "lex_item_choice", "Often_confused", "lex_part_choice", "Absence_comp_colloc", "Redundant", "Derivation", "Formational_affixes", "Suffix", "Prefix", "Category_confusion", "Compound_word", "Discourse", "Ref_device", "Coherence", "Linking_device", "Inappropriate_register", "Absence_comp_sent", "Redundant_comp", "Absence_explanation", "delete"]
translate_dict = {e[0]: e[1] for e in zip(Conventional, Tags)}

error_type = "Noun number" #@param ["Spelling", "Choice of lexical item", "Deletion", "Prepositions", "Agreement", "Noun number", "Confusion of categories", "Referential device", "Capitalisation", "Words often confused"]
error_ratio = "ToFifteen" #@param ["ToFifteen", "AugmentedRatio"]

error_type = translate_dict[error_type]

It's about time we named our model

In [9]:
modelname = "Noun_number_ToFifteen" # @param {type:"string"}

In [10]:
import shutil

filename = error_ratio+"/train/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'.')
trn = pd.read_json(error_type+".json").reset_index(drop=True)
trn["is_error"] = trn["is_error"].astype(int)

filename = error_ratio+"/test/"+error_type+".json"
shutil.copy2('./Datasets/'+filename,'./'+error_type+'_test.json')
tst = pd.read_json(error_type+"_test.json").reset_index(drop=True)
tst["is_error"] = tst["is_error"].astype(int)

In [11]:
len(trn)

37056

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [12]:
train = trn
test = tst

In [13]:
train.columns

Index(['context', 'is_error', 'path', 'substring'], dtype='object')

For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [14]:
DATA_COLUMN = 'context'
SUBSTR_COLUMN = 'substring'
LABEL_COLUMN = 'is_error'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [15]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = x[SUBSTR_COLUMN], 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [16]:
train_InputExamples

0        <bert.run_classifier.InputExample object at 0x...
1        <bert.run_classifier.InputExample object at 0x...
2        <bert.run_classifier.InputExample object at 0x...
3        <bert.run_classifier.InputExample object at 0x...
4        <bert.run_classifier.InputExample object at 0x...
5        <bert.run_classifier.InputExample object at 0x...
6        <bert.run_classifier.InputExample object at 0x...
7        <bert.run_classifier.InputExample object at 0x...
8        <bert.run_classifier.InputExample object at 0x...
9        <bert.run_classifier.InputExample object at 0x...
10       <bert.run_classifier.InputExample object at 0x...
11       <bert.run_classifier.InputExample object at 0x...
12       <bert.run_classifier.InputExample object at 0x...
13       <bert.run_classifier.InputExample object at 0x...
14       <bert.run_classifier.InputExample object at 0x...
15       <bert.run_classifier.InputExample object at 0x...
16       <bert.run_classifier.InputExample object at 0x.

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [17]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1"

class NonMaskOmittingTokenizer(bert.tokenization.FullTokenizer):
  def tokenize(self, text):
    split_tokens = []
    for token in self.basic_tokenizer.tokenize(text):
      for sub_token in self.wordpiece_tokenizer.tokenize(token):
        split_tokens.append(sub_token)

    for index, item in enumerate(split_tokens):
      if index >= len(split_tokens)-2:
        break
      if item == '[' and split_tokens[index + 1] == 'MA' and split_tokens[index + 2] == '##S' and split_tokens[index + 3] == '##K' and split_tokens[index + 4] == ']':
        split_tokens[index] = "[MASK]"
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]
        del split_tokens[index + 1]

    return split_tokens

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return NonMaskOmittingTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 09:49:32.914246 140149597505280 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [18]:
tokenizer.tokenize("This here's an example of using the BERT [MASK] tokenizer")

['This',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'B',
 '##ER',
 '##T',
 '[MASK]',
 'token',
 '##izer']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [19]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 37056


I0509 09:49:52.642787 140149597505280 tf_logging.py:115] Writing example 0 of 37056


INFO:tensorflow:*** Example ***


I0509 09:49:52.646042 140149597505280 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: None


...

I0509 09:50:30.859712 140149597505280 tf_logging.py:115] input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


I0509 09:50:30.861449 140149597505280 tf_logging.py:115] segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


I0509 09:50:30.863495 140149597505280 tf_logging.py:115] label: 0 (id = 0)


#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [20]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [21]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [22]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [23]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS) * 2
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

print(num_train_steps)

6948


In [24]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [25]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_task_type': 'worker', '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f76843f7048>, '_evaluation_master': '', '_save_checkpoints_secs': None, '_train_distribute': None, '_experimental_distribute': None, '_num_ps_replicas': 0, '_protocol': None, '_tf_random_seed': None, '_device_fn': None, '_save_checkpoints_steps': 500, '_save_summary_steps': 100, '_model_dir': 'bert_noun_number', '_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True}


I0509 09:50:40.594617 140149597505280 tf_logging.py:115] Using config: {'_task_type': 'worker', '_keep_checkpoint_max': 5, '_global_id_in_cluster': 0, '_log_step_count_steps': 100, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f76843f7048>, '_evaluation_master': '', '_save_checkpoints_secs': None, '_train_distribute': None, '_experimental_distribute': None, '_num_ps_replicas': 0, '_protocol': None, '_tf_random_seed': None, '_device_fn': None, '_save_checkpoints_steps': 500, '_save_summary_steps': 100, '_model_dir': 'bert_noun_number', '_task_id': 0, '_master': '', '_num_worker_replicas': 1, '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_is_chief': True}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [26]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

In [27]:
print('Beginning Training!')
current_time = datetime.now()

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.3)

with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    with tf.device('/gpu:0'):
        estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.


I0509 09:50:57.382159 140149597505280 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 09:51:00.905976 140149597505280 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving checkpoints for 6948 into bert_noun_number/model.ckpt.


I0509 10:36:28.118179 140149597505280 tf_logging.py:115] Saving checkpoints for 6948 into bert_noun_number/model.ckpt.


INFO:tensorflow:Loss for final step: 0.00030871131.


I0509 10:36:31.594422 140149597505280 tf_logging.py:115] Loss for final step: 0.00030871131.


Training took time  0:45:50.957153


Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

Now let's use our test data to see how well our model did:

In [28]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [35]:
# estimator.evaluate(input_fn=test_input_fn, steps=None)
estimator.evaluate(input_fn=test_input_fn, checkpoint_path="./bert_noun_number/model.ckpt-5000", steps=None)

INFO:tensorflow:Calling model_fn.


I0509 10:54:04.422664 140149597505280 tf_logging.py:115] Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0509 10:54:08.102561 140149597505280 tf_logging.py:115] Saver not created because there are no variables in the graph to restore
/home/ivantorubarov/anaconda3/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 5000: auc = 0.7807714, eval_accuracy = 0.9471071, f1_score = 0.5826235, false_negatives = 237.0, false_positives = 253.0, global_step = 5000, loss = 0.30409938, precision = 0.57478994, recall = 0.59067357, true_negatives = 8432.0, true_positives = 342.0


I0509 10:54:56.588682 140149597505280 tf_logging.py:115] Saving dict for global step 5000: auc = 0.7807714, eval_accuracy = 0.9471071, f1_score = 0.5826235, false_negatives = 237.0, false_positives = 253.0, global_step = 5000, loss = 0.30409938, precision = 0.57478994, recall = 0.59067357, true_negatives = 8432.0, true_positives = 342.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: ./bert_noun_number/model.ckpt-5000


I0509 10:54:56.590573 140149597505280 tf_logging.py:115] Saving 'checkpoint_path' summary for global step 5000: ./bert_noun_number/model.ckpt-5000


{'auc': 0.7807714,
 'eval_accuracy': 0.9471071,
 'f1_score': 0.5826235,
 'false_negatives': 237.0,
 'false_positives': 253.0,
 'global_step': 5000,
 'loss': 0.30409938,
 'precision': 0.57478994,
 'recall': 0.59067357,
 'true_negatives': 8432.0,
 'true_positives': 342.0}

# Extracting the trained model


Now let's save our model:

In [47]:
def getPrediction(in_sentences):
  labels = ["Not an error", "Is an error"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x[0], text_b = x[1], label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  # predictions = estimator.predict(predict_input_fn)
  predictions = estimator.predict(predict_input_fn, checkpoint_path="./bert_noun_number/model.ckpt-5500")
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [37]:
import nltk
import re
import itertools

nltk.download('perluniprops')
nltk.download('punkt')

[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/ivantorubarov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [38]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()

from nltk.tokenize.moses import MosesDetokenizer
detokenizer = MosesDetokenizer()

def annotate(text):
  current_time = datetime.now()
  raw = text
  raw = re.sub(r'\s', ' ', raw)
  raw = re.sub(r'( )+', ' ', raw)
  sentences = nltk.sent_tokenize(raw)
  wordsoup = []
  annotation_set = []
  checking_ids = []
  tokens = []
  J = 0
  for sentence in sentences:
    wordsoup.append(tknzr.tokenize(sentence))
  for i in range(len(wordsoup)):
    tokenized_sentence = wordsoup[i]
    for j in range(len(tokenized_sentence)):
      token = tokenized_sentence[j]
      if not re.search(r'[a-zA-Z]', token):
        tokens.append([token, 0])
      else:
        tokens.append([token, None])
        substring = token
        si = i-2
        if si < 0:
          si = 0
        detokenizing_string = wordsoup[si:i] + [tokenized_sentence[:j] + ['[MASK]'] + tokenized_sentence[j+1:]] + wordsoup[i+1:i+3]
        detokenizing_string = [item for sublist in detokenizing_string for item in sublist]
        entry = detokenizer.detokenize(detokenizing_string, return_str=True)
        annotation_set.append([entry, substring])
        checking_ids.append(J)
      J += 1
  predictions = getPrediction(annotation_set[:128])
  for p in range(len(predictions)):
    if predictions[p][-1] == 'Is an error':
      tokens[checking_ids[p]][1] = 1
    else:
      tokens[checking_ids[p]][1] = 0
  print('\n')
  print("Annotation took time ", datetime.now() - current_time)
  return tokens

def print_annotated_webpage(tokens, title):
  out = '<html>\n<head>\n<title>'+title+'</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>'
  outsoup = []
  for token in tokens:
    if token[1] == 1:
      token[0] = '<div class="blue">'+token[0]+'</div>'
    outsoup.append(token[0])
  bodystring = detokenizer.detokenize(outsoup, return_str=True)
  out += bodystring
  out += '</body>\n</html>'
  return out

In [48]:
import json

curname = "wsites_" + modelname + ".json"

test_texts = {"./data/old IELTS/IELTS2016/JSl_43_1.txt": "The given diagram illustrates how much young people in different area and both gender can not go to primary school. In addition, two years: 2000 and 2012, are presented on diagram. It can be seen that in each region and in both gender the number of children without access to primary education was decreased. The most sharp decrease was shown in South Asia from 32,7 million totally (11,1 million boys + 21,6 million girls) to 9,9 million (5,1 million boys + 4,8 million girls) in 2012 year. Furthermore, in South Asia among girls this number decreased more than 5 while among boys it consisted 2. As for Africa, this region remained with a biggest number of younger without primary education: 43,7 million totally in 2000 and 32,7 million in 2012. Moreover, in rest of world slop was not so dramatic and the number of children without access to education decreased by 8 million. By the way, there was a downward trend from 2000 to 2012 in number of young people who have not access to primary education as in Africa, South Asia and rest of world as among boys and girls.\n", "./data/exam/exam2017/VSa_80_2.txt": "The issue about the use of medical help in sport always bear a lot of controversy. The problem of medical drugs being used in sport and punishment for this will be discussed in this essay.\nSome people are concerned that the use of drug is affair of sportsmen who are who can not afford themselves to stick to the rules. But people believe that these sportsmen use the drugs for a special reason - they are adicted to the drugs as the can not compete without them. For instance, the use of medical tablets in an aim of being calm and conscentious during the competition or a game. In this situation sportsmen should not be banned from the sport for all their life only because of them being nervous, and panicked before and during the competition - such sportsmen are just in need of this help and they should not be blamed for the rest of their life.\nHowever, other people believe that if sportsmen are into taking drugs as a help in the sport, they will never stop themselves from using them even after a punishment. This idea depictes that such sportsmen are satisfied with aim of winning the sport competition in any way and who does not matter about the honesty and clearness of the sport. The main rule of such sportsmen is: \"the aim is always worth of any sources\", and the solution of a problem of how such athletes should be punished lies in them being banned for the all time and left the sport.\nTo my way of thinking, people should not use drugs in the sport at all as it is a game that should never cost in nervous system suffering or in winning in any possible ways. Sticking to this idea, I can not agree neither with the first argument, nor with the second - the sport is only a game and it should be free from the medical support. However, there always will be people who are concerned about the use of drugs as a help for sportsmen being calm during their job and about them not be banned from the sport for the rest of their life only because of this. In spite of this, other people will always state that the sportsmen who use the drug once, tend to repeat the use of it again, and for such athletes there is no place in sport in any time and after any punishment.\n", "./data/2012-2014/esl_00978.txt": "The gap between parents and their children's generations stemps from lack of knowledge and good will from both parties.\nI agree with this stament.To my mind, generation gap is a great problem of the society. And, actually, the lack of knowledge what to do or what topic speak about is bad for childrens and their parents. Adults think that are intelligent and know everything about their children's lives. They are not trying to take into account that things as fashion and slang are changing every day. So, parents and adults can't get on well with each other, because their speach and attitude are not similar. I suggest paople who has communicative problems try to read books and magazings, use the Internet to find the way to solve their communication problems. There are a lot of workshops which help people to cope with their problems. Parents should know about them. Children, in its turn, are not very patient to their relatives. They think. that adults are to old for them, So, and children and parents should study to speak with each other on topics which everyone likes.\n", "./data/exam/exam2017_6/OBy_62_1.txt": "The pie charts illustrates transportations by rail and roads in Eastern in 2008 The name of segments of goods is Machinery, Metals, Food products, Chemicals, Non-metallic minerals and Manufactured goods. The biggest segment in charts is Metals (35%) goods by rail and food product by road. But manufactured goods have a higher percent of transported by rail (25%) and by road (28%). At the chart by road machinery have the lowest percent \u2013 2%. However, in chart by rail the lowest percent have a non-metallic minerals. Another goods have a middle percent of transported by rail and road. By the way charts have a two lead goods (manufactured goods and metals by rail; food product and manufactured goods) and balanced percent another goods.\n\nIn conclusion, something goods easier transported by the rail, and something goods easier by the road. As an example, metal and machinery better transported by rail, food and non-metallic minerals by roads.\n", "./data/old IELTS/IELTS2015/EPa_6_2.txt": "Nowadays, public health is very important problem. Government should do enough to improve it. Some people consider that the best way to improve public health is increasing the number of sport facilities. Other people think that this is not effective and other measures are required.\nSport facilities is good way to solve a problem. Many people can`t afford go to gyms and free sport facilities will be appropriate. In our town, there are free sport facilities on streets and yards. Everyone can go and use it. Every person knows that sport is a life. And increase sport facilities right. Gyms can make a special offers in order to attract new clients.\nOther point of view is other measures are required. Government should prohibit unhealthy activities in public places. It should take a law, which banned to smoke and drink alcohol on streets, in parks, in stadiums. Government should provide social programs which can support public health. For example, give free medicine or doctors at work places and schools. Doctor can give public lectures about health.\nTo my mind, government should take other measures to improve public health, because sport facilities not always effective. People can have lack of time to do sports. And public health will be improved when people give up bad habits.\n", "./data/exam/exam2014/2012-2014_4/esl_00808.txt": "Nowadays, our lives are full of modern technology and many of us use many different devices. It has both positive and negative influence on human\u2019s life. Some people find it more negative. Therefore, I will consider these two points and we will see to what extent this is true.\nThere is a point of view claiming that people are failing to learn, or are forgetting, many basic skills. For reasons it is true. Many of us use modern technology to boost their everyday challenges. In other words, the devices, that we have, simplify our life. Our basic skills may suffer from that prevalence of modern technology, for example many taxi drivers or just drivers use GPS navigators, which designed to simplify the process of finding the right way or even more the shortest way. This fact leads to loss of basic skill of reading the map. This basic life skill isn\u2019t acquiring by taxi or lorry drivers, and if someday there will no GPS navigators, satellites or simply connection to them isn\u2019t good, they will fail at their job. Even basic mathematical skills are being hebetated by a genius invention \u2013 calculator. A significant amount of people use this device daily or every time when they want to count up something quickly. \nThese examples show that many people use modern technology to simplify their lives, but there are still the great number of drivers who use the time-honored map and still there are mathematicians who can count in head.\n", "./data/exam/exam2014/VKo_1_2.txt": "Government take a big part of our lifes. We have a different cases of policy making, of regims and governence. How to government should give a freedom? Government should give a total freedom or restrict some points? Now (and in our history also) it is a very big problem to governemt and cityzens. It's better to discuss the advantages and disadvantages of total freedom for the creative artist for instance. Can they make work with sens of freedom? What can restrict their?\nFirst of all discuss the opinion about restriction. In democracy we can't restrict anybody if they don't make a crime. In other political regims we can see that some artists can't work and move in other country or government make force to create according with some rools. The rools should maintain order but don't restrict people. In this case restriction have a positive hand. But government can restrict our body and our action in democracy. It is called biopolitics when government care about halfy and private life. In this way we could not notice that we are controlled. For instance government controll sexuality, smoking or alcoghol and refused to editors of film or books to use this topics.\nHowever how to explaine to somebody about sexuality or  desease, for example, if we can use this themes? The new word in arts it totally revolution. Picasso for instanse or other postmodern afrtist attract us because their art do not restricted. They don't limit themself and many people love this art. Somebody could dislike or hate modern art but they havn't a rool to restrict other people, they can't limit right of others to admire another culture, another style.\nAt the end I want to say that no goverment or nobody else can not restrict the creative artists. But artists sould understand what they do and don't declain limits of society. All people should live in peace.\n", "./data/exam/exam2016/JSl_50_2.txt": "As we can see, modern world is full of different companies and the contest of rising concurse tension some of companies deside to move their production lines to some poor countries. But does this decision really useful? Some of advantages of such kind of decisions seems really attractive. For example, such measures will bring some economical profits for the company, such as low cost of labor force, low tax rate and planty of people who are willing to work for any sum of money. Moreover, the \u201chost country\u201d benefits too. The government collect more taxes. People have waraing places. Almost, one of the advantages of such decision is including of country in modern global economicy, giving impulce to the globalisation. On the other hand the \u201cmother country\u201d of such company have less in taxes. Moreover, it almost last a huge amount of working places elevating the problem of unemployment. Modern socials are struggeling with the problems and replacing of industy just makes the thing worse. And, finally, country still have some problems in addition to benefits, for example the low rate of human rights and low level of life. And still, this country becomes dependent on foreign capital, wich can become an obstacle for development of local industry.  As we can see disadvantages outweight the advantages, and it becomes obvious in the contect of soviety benefits. Modern countries try to return industry from the outside because they lost a lot of profit. And I am totally agree with such position, as I think that the country can\u2019t be convically strong without industry. In conclusion I would like to add, that, of course, companies gain a lot from moving their business to same not-so-rich countries, but their own society have just tosses. \n", "./data/exam/exam2014/MTsy_15_2.txt": "It is certainly true that people need to improve their health all the time. One of the possible solutions is to make more sports facilities. However, other consider that it will not help to solve this problem and there will be other methods for improving public health.\nOn the one hand, increasing the number of sport facilities is a good way for making peoples health better. Boys and girls could visit different sports sections and play in their favourite games. The time spent on watching TV and sitting in front of the computers will definitely decrease. For example, when my brother began to visit the football section, he had not got a time for chatting and playing computer games. Moreover, he became stronger.\nOn the other hand, not all people have got enough money for viviting sports clubs. For instance, if the government and businessmen open more fitness clubs, there will not be much more women visiting its, because abonements are rather expensive. Furthermore, old people can not visit sport sections because of their health.\nHence, there will another methods for improving public health. I consider that the government should think about making more medical services. Increasing the number of competent doctors is another way for solving this problem. Besides, all medical tablets should be available for people.\nFrom my point of view, increasing the number of sports facilities is not enough foo improving health. Competent doctors, available tablets and good medical services are required.\n", "./data/exam/exam2017/DOv_27_1.txt": "Internet is a new Fenomenon in our world. Our generation is aboped to internet easy, but How previous generation react introduse in our live.\nOn diagram is showed persent of online adult in USA who use networks. What can we see? Firstly, the people's activity over 65 year old is lower than activity of another groups. Next, we can see that then higher age than lower share of people who use comunicatin networks (such as Facebook or instagram). But it is not true for work networks (such as Linkedin). Share People with age between 40 and 64 use work networks higher then share of othor groups. About Facebook. It is most popular network in all groups. It may be explained by different funccioning of this network.\n", "./data/exam/exam2014/VKo_7_1.txt": "The chart represents the different levels of post-school  qualification in Australia and the proportion of men and women who held them i 1999.\nThe level of men with Master's degree is 20% higher than that of women. The level of men, who held a postgraduate diploma is more than a twice as big as the level of women. However, there is a little difference between the level of men and women with Bachelor's degree. The percentage of females, who held an undergraduate diploma is two times higher than the percentage of males with an undergraduated diploma and it is about 70% and 35%. On the contrary, the level of men, who held a skilled vocational diploma in 1999 is nine times higher than the level  of women with sush post-school qualifications.\nThe chart below indicates the information about 5 different post-school qualifications. The level of males who held diploma or degree in 1999 is higher in 3 of 5 represented qualification. Which includes skilled vocational diploma, postgraduate diploma and masters degree. However, the percentage of women, who held undergraduate diploma and bachelors degree is higher, than the percentage of men with the same post-school qualifications. \n", "./data/exam/exam2017/EGe_264_1.txt": "The information given represents the highest and the lowest temperatures in every month in Yakutsk and Rio de Janeiro. The data are provided in average degree per month.\nIt can be deduced from the graph that maximum temperatures in Brazil increase from 25 degrees above 0 in June and July to approximately 30 degrees above 0 in December and January. Minimum temperatures in Rio de Janeiro go down from near 25 degrees in February to the lowest 18 degrees above 0 in July. It can be noticed that the coldest month in this town is July and the warmest is February, because its minimum and maximum temperatures are the highest.\nIn Russia the graphs of temperatures are more variable. The highest temperatures rise from 35 degrees below 0 in two winter months to 25 degrees in July. Temperature minimum declines from approximately 10 degrees above 0 in summer to near 41 degrees below 0 in winter. To conclude, the changes in temperature of the warmest month show two-time decline, at the same time temperature variaty in winter is not so noticable.\nTo sum up, it needs to compare this two graphs. Average temperatures in Rio de Janeiro remain stable all over the year. In contrast, temperatures in Yakutsk change dramatically in different seasons. Differents between minimum and maximum temperatures are approximately 10 degrees in Brazil and go from 5 to 10 degrees in Russia in each month.", "./data/old IELTS/IELTS2015/EEm_29_2.txt": "All the universities should allow the same amount of males and females to enter. This is the common statement of many people and a fair one at that.\n\nFirst of all, allowing the same amount of teenagers of both genders to enter the same university is good for the reputation of that university. Taking modern tendencies into account this kind of university would be seen democratic and most forward thinking. This university will be seen as the one which supports equall rights and gives freedom without giving priorities to any kind of students. Tis will bring university success and increase it\u2019s popularity. Secondly, having the same amount of male and female students makes it comfortable for the both groups of the student to study and express themxelves, which is not only good for educational purposes, but is once again good for the university\u2019s popularity.Making them participate in same kind of activities makes them more confident ad further improve their communication skills, which is also\u2026 Thirdly, but not less importanly having the same amount of male and female students gives them freedom to express themselves, while adding a competitive aspect to studying, which streamlines the process of gaining knowledge, making it more effective, while adding incentive for all students to participate.\n\nTo sum up, universitis, which accept equal amount of male and female students only gain points to popularity and efficiency, while leaving behind the thoughts about the difference in approach, when studying students of different genders.\n", "./data/exam/exam2017/OBy_75_2.txt": "There is a common opinion that technology we use every day can be the reason of our health problems, increasing each year. It sounds like truth because doubtless each person reveal some problems with sleep or with their eyes or problems like headache after using their gadgets.\n\nNowdays a lot of people complain the headache. Many if us have medcine from  it. And, actually, out telephones, computers, other technic we use have a big inlue. Waves from gadgets sometimes are very harmful for our organism. The solution of this problem is simple \u2013 reduction of the usage.\n\nThe second problem each person can face is problem with sleep \u2013 somnium. After long day you can think that you will fall asleep in the same moment you will lie in the bed but it is not work. The phone may be the reason of it too. The waves that were be mentioned are not healthy for us, so the way to reduce the problem is not to sleep with the phone. In addition, some people have the lond-lasting session in the Internet before, they scroll news in social nets while lying in the bed and after that have some problems with falling asleep. The advice is to switch off all gadgets and take it away from the room.\n\nOur health is very important and we should value it. We ought to control the usage of gadgets and not lot it to influe on our organism.\n", "./data/exam/exam2017/OBy_179_1.txt": "The pie charts show the percentage of goods delivered by rail and by road between countries in Eastern Europe in 2008. \n\nTo begin with, the product transported by road are the same than transported by rail. In the first pie chart we can see 35% of metals transported by rail whereas there are only 11% of metals delivered by road. In the bot charts we can see about the same number of manufactured goods (25% and 28%). Machinery is mostly transported by rail (12%), when delivery by road is only 2%. However, food products mostly delivered by road (30%). Non-metallic minerals (13%) transported by road and it is half-higher than transportation by rail. Transportation of chemicals in both charts is pretty the same. \n\nOverall, the percentage of manufactured goods are the same level in both pie charts. Metals and machinery delivered by rail mostly. Whereas food product is beneficially transported by road. \n"}
outie = [print_annotated_webpage(annotate(test_texts[tt]), tt) for tt in test_texts]
with open(curname, 'w', encoding="utf-8") as w:
  json.dump(outie, w)

INFO:tensorflow:Writing example 0 of 123


I0509 11:03:09.503220 140149597505280 tf_logging.py:115] Writing example 0 of 123


INFO:tensorflow:*** Example ***


I0509 11:03:09.506128 140149597505280 tf_logging.py:115] *** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Running local_init_op.


I0509 11:04:58.537975 140149597505280 tf_logging.py:115] Running local_init_op.


INFO:tensorflow:Done running local_init_op.


I0509 11:04:58.632358 140149597505280 tf_logging.py:115] Done running local_init_op.




Annotation took time  0:00:07.524130


In [42]:
os.listdir(".")

['.ipython',
 '.cache',
 'bert_noun_number',
 '.ipynb_checkpoints',
 'bert_prepositions',
 'delete.json',
 '.jupyter',
 '.bashrc-anaconda3.bak',
 '.config',
 'REALEC_AutoAnnotator_BERT_pipeline Often_confused.ipynb',
 'Prepositions_test.json',
 'Noun_number_test.json',
 'wsites_OftenConfused_ToFifteen.json',
 'Often_confused_test.json',
 'Often_confused.json',
 'anaconda3',
 'Anaconda3-4.0.0-Linux-x86_64.sh',
 'Agreement_errors_test.json',
 'download_google_drive',
 'REALEC_AutoAnnotator_BERT_pipeline Noun number.ipynb',
 '.nano',
 '.bash_history',
 'wsites_Prepositions_ToFifteen.json',
 'Capitalisation.json',
 'Prepositions.json',
 'wsites_Noun_number_ToFifteen.json',
 'gdrive-linux-x64',
 '.keras',
 'delete_test.json',
 '.python_history',
 'nltk_data',
 'wsites_delete_ToFifteen.json',
 'wsites_Capitalisation_ToFifteen.json',
 'Datasets',
 '.local',
 'wsites_lex_item_choice_AugmentedRatio.json',
 'REALEC_AutoAnnotator_BERT_pipeline Capitalisation.ipynb',
 'jupyter-drive',
 'cuda-repo-

In [49]:
outie[4]

'<html>\n<head>\n<title>./data/exam/exam2014/VKo_1_2.txt</title>\n<meta charset="utf-8">\n<style type="text/css">\n.blue {\n\tbackground: #a8d1ff;\n\tdisplay: inline-block;\n}\n</style>\n</head>\n<body>Government take a big part of our <div class="blue">lifes</div>. We have a different cases of policy making, of regims and governence. How to government should give a freedom? Government should give a total freedom or restrict some points? Now ( and in our history also) it is a very big problem to governemt and cityzens. It\'s better to discuss the advantages and disadvantages of total freedom for the creative artist for instance. Can they make work with sens of freedom? What can restrict their? First of all discuss the opinion about <div class="blue">restriction</div>. In democracy we can\'t restrict anybody if they don\'t make a crime. In other political regims we can see that some artists can\'t work and move in other <div class="blue">country</div> or government make force to create 

In [50]:
!./gdrive-linux-x64 upload wsites_Noun_number_ToFifteen.json

Uploading wsites_Noun_number_ToFifteen.json
Uploaded 1npxRo0xotjlXIYJLRMW72UNsPtqqiCuT at 24.1 KB/s, total 24.1 KB


In [41]:
!ls -lAh ./bert_noun_number

total 7.0G
-rw-rw-r-- 1 ivantorubarov ivantorubarov  271 May  9 10:36 checkpoint
drwxr-xr-x 2 ivantorubarov ivantorubarov 4.0K May  9 10:46 eval
-rw-rw-r-- 1 ivantorubarov ivantorubarov 119M May  9 09:41 events.out.tfevents.1557394855.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 239M May  9 10:36 events.out.tfevents.1557395484.instance-2
-rw-rw-r-- 1 ivantorubarov ivantorubarov 225M May  9 09:51 graph.pbtxt
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 10:24 model.ckpt-5000.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 10:24 model.ckpt-5000.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  61M May  9 10:24 model.ckpt-5000.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 10:27 model.ckpt-5500.data-00000-of-00001
-rw-rw-r-- 1 ivantorubarov ivantorubarov  23K May  9 10:27 model.ckpt-5500.index
-rw-rw-r-- 1 ivantorubarov ivantorubarov  61M May  9 10:27 model.ckpt-5500.meta
-rw-rw-r-- 1 ivantorubarov ivantorubarov 1.3G May  9 10:30 model.ckpt-

In [ ]:
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.data-00000-of-00001
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.index
!./gdrive-linux-x64 upload ./bert_output/model.ckpt-2396.meta

Uploading ./bert_output/model.ckpt-2396.data-00000-of-00001
234.9 MB/1.3 GB, Rate: 41.1 MB/s                  

In [1]:
!rm -rf ./bert_output

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': curname})
uploaded.SetContentFile('./'+curname)
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 16_pmUKOHS5lKXM2o-1QyVcPQohalCRZd


# Preparing existing model for REALEC production